In [1]:
import pandas as pd
import glob
import numpy as np
from tqdm import tqdm
import imblearn
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from collections import Counter
#from dask.dataframe import from_pandas
#import dask.dataframe as dd
training_val_combined=pd.read_parquet('/content/ECE_Project_C1/combined_sampled/training.gzip', engine='auto')
training_val_combined=training_val_combined.reset_index()
training_val_combined=training_val_combined[['measurements','labels']]
training_val_combined['labels']=training_val_combined['labels'].astype(int)
from sklearn.preprocessing import OneHotEncoder
# define one hot encoding

In [1]:
!git clone https://github.com/mfarooq2/ECE_Project_C1.git

Cloning into 'ECE_Project_C1'...
remote: Enumerating objects: 311, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 311 (delta 17), reused 66 (delta 17), pack-reused 245
Receiving objects: 100% (311/311), 256.52 MiB | 24.10 MiB/s, done.
Resolving deltas: 100% (130/130), done.
Updating files: 100% (197/197), done.


In [2]:
X=np.vstack(training_val_combined['measurements'])
y=training_val_combined['labels']
cro=Counter(y)
print(cro)
y=np.vstack(y)
org1=list(cro.items())
org=list(cro.items())
org=[(j,i) for i,j in org]
org.sort()
new=[(j,((max(cro.values())-i))) for i,j in org]
new.sort()
new=dict(new)
new[0]=max(cro.values())
over_samp = imblearn.over_sampling.RandomOverSampler(sampling_strategy=new)
X_balanced,y_balanced=over_samp.fit_resample(X,y)

Counter({0: 1006926, 3: 206436, 2: 73068, 1: 55216})


In [3]:
idx = np.random.permutation(len(X_balanced))
X_balanced_reshaped = X_balanced[idx].reshape(len(X_balanced),3,2,1)
y_balanced_reshaped = y_balanced[idx].reshape(-1,1)

In [4]:
num_classes=len(np.unique(y_balanced_reshaped))

In [5]:
encoder = OneHotEncoder()
# transform data
onehot = encoder.fit_transform(y_balanced_reshaped)
y_balanced_reshaped_onehot=onehot.toarray()

In [14]:
y_balanced_reshaped_onehot

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]])

In [6]:
def make_model(input_shape,num_classes=num_classes):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv2D(filters=64, kernel_size=2, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv2D(filters=64, kernel_size=2, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv2D(filters=64, kernel_size=2, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling2D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="relu")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

In [7]:
model = make_model(input_shape=X_balanced_reshaped.shape[1:],num_classes=num_classes)
epochs = 50
batch_size = 250

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [33]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 3, 2, 1)]         0         
                                                                 
 conv2d_12 (Conv2D)          (None, 3, 2, 64)          320       
                                                                 
 batch_normalization_12 (Bat  (None, 3, 2, 64)         256       
 chNormalization)                                                
                                                                 
 re_lu_12 (ReLU)             (None, 3, 2, 64)          0         
                                                                 
 conv2d_13 (Conv2D)          (None, 3, 2, 64)          16448     
                                                                 
 batch_normalization_13 (Bat  (None, 3, 2, 64)         256       
 chNormalization)                                          

In [8]:
history = model.fit(
    X_balanced_reshaped,
    y_balanced_reshaped_onehot,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

Epoch 1/50
11818/11818 [==============================] - 76s 6ms/step - loss: 1.1907 - accuracy: 0.4554 - val_loss: 1.1392 - val_accuracy: 0.5141 - lr: 0.0010
Epoch 2/50
11818/11818 [==============================] - 74s 6ms/step - loss: 1.1298 - accuracy: 0.5001 - val_loss: 1.1467 - val_accuracy: 0.4871 - lr: 0.0010
Epoch 3/50
11818/11818 [==============================] - 74s 6ms/step - loss: 1.1079 - accuracy: 0.5143 - val_loss: 1.0332 - val_accuracy: 0.5577 - lr: 0.0010
Epoch 4/50
11818/11818 [==============================] - 75s 6ms/step - loss: 1.0367 - accuracy: 0.5559 - val_loss: 1.0197 - val_accuracy: 0.5606 - lr: 0.0010
Epoch 5/50
11818/11818 [==============================] - 74s 6ms/step - loss: 1.0119 - accuracy: 0.5659 - val_loss: 1.2026 - val_accuracy: 0.5452 - lr: 0.0010
Epoch 6/50
11818/11818 [==============================] - 74s 6ms/step - loss: 1.0708 - accuracy: 0.5396 - val_loss: 1.0237 - val_accuracy: 0.5674 - lr: 0.0010
Epoch 7/50
11818/11818 [================

In [ ]:
X_balanced.shape[1:]

In [ ]:
input_shape=X_balanced_reshaped.shape[1:]

In [ ]:
input_layer = keras.layers.Input(input_shape)

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn import svm
import pandas as pd
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, confusion_matrix, ConfusionMatrixDisplay
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
def data_frame_adder(temp_frame,file_name):
    file_name=file_name.split('.csv')[0]
    temp_frame['file_name']=file_name
    temp_frame['subject']=temp_frame['file_name'].apply(lambda x: int(x.split('subject_')[1].split('_')[0]))
    temp_frame['session']=temp_frame['file_name'].apply(lambda x: int(x.split('subject_')[1].split('_')[1]))
    #temp_frame['type']=temp_frame['file_name'].apply(lambda x: x.split('__')[1])
    temp_frame = temp_frame.drop(columns=['file_name'])
    temp_frame = temp_frame[temp_frame.columns[::-1]]
    temp_frame = temp_frame.sort_values(by=['session','subject'], axis=0)
    col_list1 = list(temp_frame.columns)[0:2]
    col_list2 = list(temp_frame.columns)[2:]
    col_list2.sort()
    col_list=col_list1+col_list2
    
    #temp_frame = swap_columns(temp_frame, 'subject', 'session')
    # Swapping the columns
    temp_frame = temp_frame.reindex(columns=col_list)

    return temp_frame
Training_data_files=glob.glob('/content/ECE_Project_C1/TestData/*.csv')
file_name_list_x=[]
file_name_list_x_time=[]
file_name_list_y=[]
file_name_list_y_time=[]
for file_name in tqdm(Training_data_files):
    if 'x.csv' in file_name:
        file_name_list_x.append(file_name)
    elif 'x_time.csv' in file_name:
        file_name_list_x_time.append(file_name)
    elif 'y.csv' in file_name:
        file_name_list_y.append(file_name)
    else:
        file_name_list_y_time.append(file_name)
file_name_list_x.sort()
file_name_list_x_time.sort()
file_name_list_y.sort()
file_name_list_y_time.sort()
Subject_list= [f.split('_')[3] for f in file_name_list_x]
Session_list= [f.split('_')[4] for f in file_name_list_x]

x_sensor_dat=pd.DataFrame([])
y_sensor_dat=pd.DataFrame([])
x_time_dat=[]



for sub,sess in tqdm(zip(Subject_list,Session_list)):
    #x_sensor_dat=pd.DataFrame(x_sensor_dat.append(pd.read_csv(f"/content/ECE_Project_C1/TrainingData/subject_{sub}_{sess}__x.csv",header=None)))
    
    x_sensor_dat=pd.concat([pd.read_csv(f"/content/ECE_Project_C1/TestData/subject_{sub}_{sess}__x.csv",header=None),pd.read_csv(f"/content/ECE_Project_C1/TestData/subject_{sub}_{sess}__x_time.csv",header=None).rename(columns={0:'time_stamp'})],axis=1)
    features_to_normalize = [0,1,2,3,4,5]
    features_to_normalize = [0,1,2,3,4,5]
    scaler = StandardScaler()
    scaler.fit(x_sensor_dat[features_to_normalize])
    scaled = scaler.fit_transform(x_sensor_dat[features_to_normalize])
    scaled_df = pd.DataFrame(scaled, columns=x_sensor_dat[features_to_normalize].columns)
    x_sensor_dat[features_to_normalize]=scaled_df

#    x_sensor_dat[features_to_normalize] = x_sensor_dat[features_to_normalize].apply(lambda x:(x-x.mean())/ x.std(), axis=0)
    #apply(lambda x:(x-x.min())/(x.max()-x.min()))
    x_sensor_dat['measurements']=x_sensor_dat.apply(lambda x: [x[0],x[1],x[2],x[3],x[4],x[5]],axis=1)
    x_sensor_dat=x_sensor_dat[['time_stamp','measurements']]
    #x_sensor_dat=x_sensor_dat.set_index('time_stamp').asfreq('0.025S')
    x_sensor_dat['Subject']=int(sub)
    x_sensor_dat['Session']=int(sess)
    #x_sensor_dat=x_sensor_dat.set_index('time_stamp').asfreq('0.025S')
    
    y_sensor_dat=pd.concat([pd.read_csv(f"/content/ECE_Project_C1/TestData/subject_{sub}_{sess}__y.csv",header=None),pd.read_csv(f"/content/ECE_Project_C1/TestData/subject_{sub}_{sess}__y_time.csv",header=None).rename(columns={0:'time_stamp'})],axis=1).rename(columns={0:'labels'})
    #y_sensor_dat['labels']=y_sensor_dat.apply(lambda y: [y[0],y[1],y[2],y[3]],axis=1)
    y_sensor_dat=y_sensor_dat[['time_stamp','labels']]
    #y_sensor_dat["time_stamp"]=pd.to_datetime(y_sensor_dat["time_stamp"],unit='s').round('ms')
    trp=pd.concat(
        [pd.DataFrame({'labels':0},index=[x_sensor_dat.index[-1]]),pd.DataFrame({'time_stamp':np.max(x_sensor_dat['time_stamp'])},index=[x_sensor_dat.index[-1]])
                         ]
         ,axis=1)
    y_sensor_dat=pd.concat([y_sensor_dat,trp],axis=0)
    trp=pd.concat([pd.DataFrame({'labels':0},index=[x_sensor_dat.index[0]]),pd.DataFrame({'time_stamp':np.min(x_sensor_dat['time_stamp'])},index=[x_sensor_dat.index[0]])],axis=1)
    y_sensor_dat=pd.concat([trp,y_sensor_dat],axis=0)
    # y_sensor_dat=y_sensor_dat.set_index('time_stamp').asfreq('0.025S')
    #y_sensor_dat['Subject']=int(sub)
    #y_sensor_dat['Session']=int(sess)
    x_sensor_dat["time_stamp"]=pd.to_datetime(x_sensor_dat["time_stamp"],unit='s').round('ms')
    x_sensor_dat=x_sensor_dat.set_index('time_stamp')
    x_sensor_dat=x_sensor_dat.asfreq('0.025S')
    
    y_sensor_dat["time_stamp"]=pd.to_datetime(y_sensor_dat["time_stamp"],unit='s').round('ms')
    y_sensor_dat=y_sensor_dat.set_index('time_stamp')
    y_sensor_dat=y_sensor_dat.asfreq(freq='0.025S', method='bfill')
    x_y_dat=pd.merge(x_sensor_dat,y_sensor_dat, how='inner', left_index=True, right_index=True)
    x_y_dat.to_parquet(f"/content/ECE_Project_C1/pre_processed_testing/subject_{sub}_session_{sess}.gzip",compression='gzip')

100%|██████████| 16/16 [00:00<00:00, 25751.67it/s]
0it [00:00, ?it/s]<ipython-input-32-75d973d105a4>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_sensor_dat['Subject']=int(sub)
<ipython-input-32-75d973d105a4>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_sensor_dat['Session']=int(sess)
<ipython-input-32-75d973d105a4>:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [35]:
x_y_dat

,measurements,Subject,Session,labels
time_stamp,,,,
1970-01-01 00:00:00.000,"[-0.16468036388457127, 0.15121226695542847, -0...",12,1,0.0
1970-01-01 00:00:00.025,"[-0.16865766837424867, 0.14967247569488737, -0...",12,1,NaN
1970-01-01 00:00:00.050,"[-0.16438116301233216, 0.1489817253354086, -0....",12,1,NaN
1970-01-01 00:00:00.075,"[-0.1552908828668455, 0.1544287369195729, -0.4...",12,1,NaN
1970-01-01 00:00:00.100,"[-0.16526145540900228, 0.15121226695542847, -0...",12,1,NaN
...,...,...,...,...
1970-01-01 00:18:52.850,"[0.159406356135594, 0.16277825502985702, -0.47...",12,1,NaN
1970-01-01 00:18:52.875,"[0.16483386876811393, 0.1645012814504027, -0.4...",12,1,NaN
1970-01-01 00:18:52.900,"[0.17994221577331065, 0.15910908643511365, -0....",12,1,NaN


In [34]:
files

[]

In [37]:

files=glob.glob('/content/ECE_Project_C1/pre_processed_testing/*.gzip')
total_training_data=pd.DataFrame([])
for f in tqdm(files):
    total_training_data=pd.concat([total_training_data,pd.read_parquet(f, engine='auto')],axis=0)
total_training_data=total_training_data.sort_values(['Subject','Session','time_stamp'])
total_training_data=total_training_data.reset_index()
total_training_data['epoch'] = total_training_data['time_stamp'].sub(pd.Timestamp('1970-01-01 00:00:00.000'))
total_training_data['epoch']=total_training_data['epoch'].dt.total_seconds()
total_training_data=total_training_data[['Subject','Session','epoch','measurements','labels']]
total_training_data_grouped=total_training_data.groupby(['Subject','Session','epoch']).agg({'measurements':[np.mean],'labels':[np.mean]})
total_training_data_grouped.columns = total_training_data_grouped.columns.droplevel(1)
total_training_data_grouped=total_training_data_grouped.reset_index(level=['epoch'])
total_training_data_grouped.to_parquet(f"/content/ECE_Project_C1/combined_sampled/testing.gzip",compression='gzip')
testing_combined=pd.read_parquet('/content/ECE_Project_C1/combined_sampled/testing.gzip', engine='auto')
testing_combined=testing_combined.reset_index()
testing_combined=testing_combined[['measurements','labels']]
X_test=np.vstack(testing_combined['measurements']).reshape(-1,3,2,1)

100%|██████████| 4/4 [00:00<00:00, 26.24it/s]


In [38]:
total_training_data

,Subject,Session,epoch,measurements,labels
0,9,1,0.000,"[-0.03765202030266226, 0.09368572626400568, -1...",0.0
1,9,1,0.025,"[-0.03524098598251466, 0.09306726091291935, -1...",NaN
2,9,1,0.050,"[-0.03380178732749251, 0.0932526808924517, -1....",NaN
3,9,1,0.075,"[-0.031089793317064792, 0.0952962881937525, -1...",NaN
4,9,1,0.100,"[-0.025079759587105822, 0.09672302515454348, -...",NaN
...,...,...,...,...,...
184147,12,1,1132.850,"[0.159406356135594, 0.16277825502985702, -0.47...",NaN
184148,12,1,1132.875,"[0.16483386876811393, 0.1645012814504027, -0.4...",NaN
184149,12,1,1132.900,"[0.17994221577331065, 0.15910908643511365, -0....",NaN
184150,12,1,1132.925,"[0.17649126596710446, 0.16459274505127874, -0....",0.0


In [39]:
preds = model.predict(X_test)

5755/5755 [==============================] - 11s 2ms/step


In [44]:
preds

array([[28.294016  ,  0.        ,  0.1563804 ,  0.32523584],
       [28.328007  ,  0.        ,  0.15285288,  0.32202303],
       [28.243238  ,  0.        ,  0.17653017,  0.36101878],
       ...,
       [23.74455   ,  0.08628803,  0.90547514,  0.02983642],
       [23.563704  ,  0.09742635,  0.9383375 ,  0.07063329],
       [23.235762  ,  0.18391532,  0.98021066,  0.09953779]],
      dtype=float32)

In [42]:
X_test.shape

(184152, 3, 2, 1)